In [5]:
from discussion_agents.cog.agent.critic import CriticAgent
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper

import os
import dotenv
dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY") 
google_cse_id = os.getenv('GOOGLE_CSE_ID')

llm = ChatOpenAI(openai_api_key=openai_api_key)
search = GoogleSearchAPIWrapper(google_api_key=google_api_key, google_cse_id=google_cse_id)

In [6]:
agent = CriticAgent(llm, search)

In [ ]:
question = ""

agent.generate(
    question=question,
    examples=,
    prompt=,
    critique_examples=,
    critique_prompt=,
)